In [1]:
import pandas as pd
from GPSat import get_data_path, get_parent_path
from GPSat.local_experts import LocalExpertOI, get_results_from_h5file
from GPSat.postprocessing import smooth_hyperparameters, SmoothingConfig
from GPSat.config_dataclasses import (DataConfig, 
                                                       ModelConfig,
                                                       PredictionLocsConfig,
                                                       ExpertLocsConfig,
                                                       RunConfig,
                                                       ExperimentConfig)


2023-06-13 11:51:44.772651: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 11:51:44.929588: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-13 11:51:45.439901: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:/usr/local/cuda-11.4/lib64
2023-06-13 11:51:45.439961: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; d

# Set up data/model/location configs

In [2]:
data_config = DataConfig(data_source=get_data_path("example", "ABC_binned_50km.h5"),
                         table="data",
                         obs_col="obs",
                         coords_col=["x", "y", "t"],
                         local_select=[{"col": "t", "comp": "<=", "val": 4},
                                       {"col": "t", "comp": "<=", "val": -4},
                                       {"col": ["x", "y"], "comp": "<", "val": 300_000}],
                         global_select=[{"col": "lat", "comp": ">=", "val": 60},
                                        {"loc_col": "t", "src_col": "date", "func": "lambda x,y: np.datetime64(pd.to_datetime(x+y, unit='D'))"}]
                        )


model_config = ModelConfig(oi_model="GPflowGPRModel",
                           init_params={"coords_scale": [50_000, 50_000, 1],
                                        "noise_variance": 0.006},
                           constraints={"lengthscales": {"low": [1e-08, 1e-08, 1e-08],
                                                         "high": [600_000, 600_000, 9]}
                                    },
                           optim_kwargs={"fixed_params": ['likelihood_variance']}
                           )


pred_locs_config = PredictionLocsConfig(method="from_dataframe",
                                       df_file=get_data_path("locations", "2d_xy_grid_5x5km.csv"),
                                       max_dist=200_000
                                       )


expert_locs_config = ExpertLocsConfig(source=get_data_path("locations", "example_expert_locations_arctic.csv"),
                                     add_data_to_col={"date": ["2020-03-04", "2020-03-05", "2020-03-06"]},
                                     col_funcs={"date": {"func": "lambda x: x.astype('datetime64[D]')",
                                                         "col_args": "date"
                                                      },
                                                "t": {"func": "lambda x: x.astype('datetime64[D]').astype(float)",
                                                      "col_args": "date"}
                                             },
                                     col_select=["x", "y", "t", "date", "lon", "lat"],
                                     row_select=[{"col": "date", "comp": "==", "val": "2020-03-05"},
                                                 {"col": "lat", "comp": ">=", "val": 60}],
                                     sort_by="date"
                                    )



In [3]:
run_config = RunConfig(store_path=get_parent_path("results", "example", "ABC_50km_test_2.h5"),
                       predict=False)

local_expert_config = ExperimentConfig(data_config,
                                        model_config,
                                        expert_locs_config,
                                        pred_locs_config)


# Train model

Once you specify the config classes, training is easy.

In [4]:
# TODO: include train only option + fixed params option
locexp = LocalExpertOI(local_expert_config=local_expert_config)
locexp.run(**run_config.to_dict())

'data_select': 0.000 seconds
'load': 0.018 seconds
current config matched previous - idx: 1
---------
storing expert locations in 'expert_locs' table
closing
'data_select': 0.019 seconds
'load': 0.021 seconds
for table: expert_locs returning 0 / 363 entries
---------
dropping expert locations that already exists in 'run_details' table
closing
'data_select': 0.020 seconds
'load': 0.021 seconds
for table: run_details returning 0 / 363 entries
'run': 0.198 seconds


In [5]:
# See the tables in the results file
with pd.HDFStore(run_config.store_path, mode="r") as store:
    all_keys = store.keys()
    print(f"Columns = {all_keys}")

Columns = ['/expert_locs', '/expert_locs_SMOOTHED', '/kernel_variance', '/kernel_variance_SMOOTHED', '/lengthscales', '/lengthscales_SMOOTHED', '/likelihood_variance', '/likelihood_variance_SMOOTHED', '/oi_config', '/oi_config_SMOOTHED', '/preds_SMOOTHED', '/run_details', '/run_details_SMOOTHED']


In [6]:
dfs, oi_configs = get_results_from_h5file(results_file=run_config.store_path,
                                          global_col_funcs=None,
                                          merge_on_expert_locations=False,
                                          select_tables=["lengthscales", "likelihood_variance", "kernel_variance"]
                                         )

issuing getting oi_config
reading in results
selecting only tables: ['lengthscales', 'likelihood_variance', 'kernel_variance']
in get_results_from_h5file trying read expert_locations from file got Exception:
No objects to concatenate
expert_locations data will not be merged on results data


/home/so/miniconda3/envs/pyOI/lib/python3.8/site-packages/tables/group.py:1160: UserWarning: problems loading leaf ``/oi_config/table``::

  HDF5 error back trace

  File "H5Dio.c", line 179, in H5Dread
    can't read data
  File "H5VLcallback.c", line 2011, in H5VL_dataset_read
    dataset read failed
  File "H5VLcallback.c", line 1978, in H5VL__dataset_read
    dataset read failed
  File "H5VLnative_dataset.c", line 166, in H5VL__native_dataset_read
    can't read data
  File "H5Dio.c", line 545, in H5D__read
    can't read data
  File "H5Dchunk.c", line 2549, in H5D__chunk_read
    unable to read raw data chunk
  File "H5Dchunk.c", line 3904, in H5D__chunk_lock
    data pipeline read failed
  File "H5Z.c", line 1400, in H5Z_pipeline
    filter returned failure during read
  File "H5Zdeflate.c", line 118, in H5Z__filter_deflate
    inflate() failed

End of HDF5 error back trace

Problems reading the array data.

The leaf will become an ``UnImplemented`` node.
  warnings.warn(


In [7]:
# Plot trained hyperparameters here


# Smooth hyperparameters

In [8]:
out_file = get_parent_path("results", "example", "ABC_50km_test_SMOOTHED_2.h5") # Path to store smoothed hyperparameters
# smooth_configs = {"lengthscales": SmoothingConfig(l_x=200_000, l_y=200_000, max=12),
#                   "likelihood_variance": SmoothingConfig(l_x=200_000, l_y=200_000),
#                   "kernel_variance": SmoothingConfig(l_x=200_000, l_y=200_000, max=0.1)}

# smooth_hyperparameters(result_file=run_config.store_path,
#                        params_to_smooth=["lengthscales", "likelihood_variance", "kernel_variance"],
#                        smooth_config_dict=smooth_configs,
#                        output_file=out_file,
#                        save_config_file=False)

smooth_configs = {"lengthscales": SmoothingConfig(l_x=200_000, l_y=200_000, max=12),
                  "kernel_variance": SmoothingConfig(l_x=200_000, l_y=200_000, max=0.1)}

smooth_hyperparameters(result_file=run_config.store_path,
                       params_to_smooth=["lengthscales", "kernel_variance"],
                       l_x = 200_000,
                       l_y = 200_000,
                       max = [12, 0.1],
                       output_file=out_file,
                       save_config_file=False)


2023-06-13 11:51:58.526824: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 11:52:01.348675: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:0 with 6250 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:17:00.0, compute capability: 7.5
2023-06-13 11:52:01.349282: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:1 with 5424 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:65:00.0, compute capability: 7.5
2023-06-13 11:52:01.355800: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6250 MB memory:  -> device: 0, name: N

found GPU
setting lengthscales to: [1.]
'__init__': 3.025 seconds
reading in results
selecting only tables: ['lengthscales', 'kernel_variance', 'likelihood_variance', 'expert_locs', 'oi_config']
expert_locations data will not be merged on results data
adding smoothed table: lengthscales_SMOOTHED
adding smoothed table: kernel_variance_SMOOTHED
copying table: likelihood_variance to likelihood_variance_SMOOTHED
writing (smoothed) hyper parameters to:
/home/so/Documents/Projects/GPSat/results/example/ABC_50km_test_SMOOTHED_2.h5
table_suffix:_SMOOTHED
writing: lengthscales_SMOOTHED to table
writing: kernel_variance_SMOOTHED to table
writing: likelihood_variance_SMOOTHED to table


In [9]:
# See the tables in the results file after smoothing
with pd.HDFStore(out_file, mode="r") as store:
    all_keys = store.keys()
    print(f"Columns = {all_keys}")

Columns = ['/kernel_variance_SMOOTHED', '/lengthscales_SMOOTHED', '/likelihood_variance_SMOOTHED']


In [10]:
dfs, oi_configs = get_results_from_h5file(results_file=out_file,
                                          global_col_funcs=None,
                                          merge_on_expert_locations=False,
                                          select_tables=["lengthscales", "likelihood_variance", "kernel_variance"],
                                          table_suffix="_SMOOTHED",
                                          add_suffix_to_table=True)

issuing getting oi_config_SMOOTHED
reading in results
selecting only tables: ['lengthscales_SMOOTHED', 'likelihood_variance_SMOOTHED', 'kernel_variance_SMOOTHED']
in get_results_from_h5file trying read expert_locations from file got Exception:
No objects to concatenate
expert_locations data will not be merged on results data


In [13]:
dfs['lengthscales_SMOOTHED']

,x,y,t,_dim_0,lengthscales
0,-1600000.0,1000000.0,18326.0,0.0,2.335665
1,-1800000.0,1000000.0,18326.0,0.0,2.537793
2,1600000.0,800000.0,18326.0,0.0,10.864260
3,1400000.0,800000.0,18326.0,0.0,9.422593
4,1200000.0,800000.0,18326.0,0.0,7.868818
...,...,...,...,...,...
1060,1400000.0,-400000.0,18326.0,2.0,1.023551
1061,1200000.0,-400000.0,18326.0,2.0,1.014111
1062,1000000.0,-400000.0,18326.0,2.0,1.001974
1063,600000.0,-400000.0,18326.0,2.0,0.990117


In [10]:
# Plot smoothed hyperparameters here

# Make predictions

To make predictions from the smoothed parameters, we re-run with a slightly different config

In [9]:
local_expert_config.model_config.load_params = {"file": out_file, "table_suffix": "_SMOOTHED"}
locexp = LocalExpertOI(local_expert_config=local_expert_config)

run_config.optimise = False
run_config.predict = True
run_config.table_suffix = "_SMOOTHED"

locexp.run(**run_config.to_dict())

'data_select': 0.000 seconds
'load': 0.013 seconds
---------
storing expert locations in 'expert_locs' table
exception occurred: 'No object named expert_locs_SMOOTHED in the file'
will now close object

---------
dropping expert locations that already exists in 'run_details' table
exception occurred: 'No object named run_details_SMOOTHED in the file'
will now close object

------------------------------
1 / 363
          x          y        t       date        lon        lat
0 -800000.0 -2000000.0  18326.0 2020-03-05 -21.801409  70.615022
'data_select': 0.023 seconds
'load': 0.023 seconds
'_update_global_data': 0.024 seconds
'local_data_select': 0.002 seconds
number obs: 0
------------------------------
2 / 363
             x          y        t       date         lon        lat
246 -1600000.0  1000000.0  18326.0 2020-03-05 -122.005383  73.040728
'_update_global_data': 0.001 seconds
'local_data_select': 0.002 seconds
number obs: 20
'_max_dist_bool': 0.013 seconds
'_from_dataframe': 0.4

2023-06-05 17:57:20.778018: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:0 with 6261 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:17:00.0, compute capability: 7.5
2023-06-05 17:57:20.779069: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:1 with 4157 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:65:00.0, compute capability: 7.5


'_read_params_from_file': 0.081 seconds
'set_parameters': 0.009 seconds
'load_params': 0.090 seconds
'set_lengthscales_constraints': 0.005 seconds
*** not optimising parameters
'get_parameters': 0.004 seconds
parameters:
lengthscales: array([2.33566515, 2.69619435, 1.00027224]) 
kernel_variance: 0.014142442820670495
likelihood_variance: 0.006000000000000003
'predict': 0.098 seconds
total run time : 1.21 seconds
------------------------------
3 / 363
             x          y        t       date         lon        lat
245 -1800000.0  1000000.0  18326.0 2020-03-05 -119.054604  71.477702
'_update_global_data': 0.001 seconds
'local_data_select': 0.002 seconds
number obs: 9
'_max_dist_bool': 0.013 seconds
'_from_dataframe': 0.015 seconds
found GPU
setting lengthscales to: [1. 1. 1.]
'__init__': 0.160 seconds


2023-06-05 17:57:21.522960: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:0 with 6261 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:17:00.0, compute capability: 7.5
2023-06-05 17:57:21.524232: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:1 with 4157 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:65:00.0, compute capability: 7.5


'_read_params_from_file': 0.098 seconds
'set_parameters': 0.006 seconds
'load_params': 0.105 seconds
'set_lengthscales_constraints': 0.006 seconds
*** not optimising parameters
'get_parameters': 0.010 seconds
parameters:
lengthscales: array([2.5377929 , 3.85636261, 1.00007004]) 
kernel_variance: 0.009260775720745573
likelihood_variance: 0.006000000000000003
'predict': 0.056 seconds
total run time : 0.71 seconds
------------------------------
4 / 363
             x          y        t       date         lon        lat
244 -2000000.0  1000000.0  18326.0 2020-03-05 -116.565051  69.868945
'_update_global_data': 0.001 seconds
'local_data_select': 0.002 seconds
number obs: 0
------------------------------
5 / 363
             x         y        t       date         lon        lat
243  1600000.0  800000.0  18326.0 2020-03-05  116.565051  73.927516
'_update_global_data': 0.001 seconds
'local_data_select': 0.002 seconds
number obs: 11
'_max_dist_bool': 0.013 seconds
'_from_dataframe': 0.015 sec

...

total run time : 0.67 seconds
------------------------------
362 / 363
            x         y        t       date        lon        lat
113  600000.0 -400000.0  18326.0 2020-03-05  56.309932  83.540179
'_update_global_data': 0.001 seconds
'local_data_select': 0.005 seconds
number obs: 33
'_max_dist_bool': 0.028 seconds
'_from_dataframe': 0.031 seconds
found GPU
setting lengthscales to: [1. 1. 1.]
'__init__': 0.137 seconds


2023-06-05 18:01:30.733427: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:0 with 6261 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:17:00.0, compute capability: 7.5
2023-06-05 18:01:30.734615: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:1 with 4157 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:65:00.0, compute capability: 7.5


'_read_params_from_file': 0.079 seconds
'set_parameters': 0.006 seconds
'load_params': 0.086 seconds
'set_lengthscales_constraints': 0.005 seconds
*** not optimising parameters
'get_parameters': 0.004 seconds
parameters:
lengthscales: array([4.84967239, 8.12480156, 0.99011742]) 
kernel_variance: 0.002399581488755304
likelihood_variance: 0.006000000000000003
'predict': 0.047 seconds
total run time : 0.66 seconds
------------------------------
363 / 363
            x          y        t       date         lon       lat
362 -200000.0  2600000.0  18326.0 2020-03-05 -175.601295  66.47575
'_update_global_data': 0.001 seconds
'local_data_select': 0.002 seconds
number obs: 6
'_max_dist_bool': 0.014 seconds
'_from_dataframe': 0.016 seconds
found GPU
setting lengthscales to: [1. 1. 1.]
'__init__': 0.171 seconds


2023-06-05 18:01:31.398402: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:0 with 6261 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:17:00.0, compute capability: 7.5
2023-06-05 18:01:31.399499: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:1 with 4157 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:65:00.0, compute capability: 7.5


'_read_params_from_file': 0.086 seconds
'set_parameters': 0.007 seconds
'load_params': 0.093 seconds
'set_lengthscales_constraints': 0.006 seconds
*** not optimising parameters
'get_parameters': 0.004 seconds
parameters:
lengthscales: array([5.71230327, 6.79157564, 1.00012475]) 
kernel_variance: 0.0006192574749181555
likelihood_variance: 0.006000000000000003
'predict': 0.049 seconds
total run time : 0.70 seconds
storing any remaining tables
SAVING RESULTS
run_details
preds
lengthscales
kernel_variance
likelihood_variance
'run': 252.515 seconds


In [ ]:
# Plot predictions